In [3]:
import csv
import ast
import numpy as np
from collections import Counter
from sklearn import preprocessing
from sklearn.metrics import recall_score
from sklearn.metrics import accuracy_score
from sklearn.preprocessing import OneHotEncoder

In [2]:
number = 0
train_path_geo =  "Voc2kGeometric/Training_data_"+str(number)+".csv"
test_path_geo =  "Voc2kGeometric/Testing_data_"+str(number)+".csv"
valid_path_geo =  "Voc2kGeometric/Validation_data_"+str(number)+".csv"
        
#Read the list 
with open(train_path_geo, 'r') as f1:
    reader = csv.reader(f1)
    your_list = list(reader)
f1.close()

#Read the list 
with open(test_path_geo, 'r') as f1:
    reader = csv.reader(f1)
    your_list_test = list(reader)
f1.close()

#Read the list 
with open(valid_path_geo, 'r') as f1:
    reader = csv.reader(f1)
    your_list_valid = list(reader)
f1.close()

In [23]:
enc = OneHotEncoder()
enc.fit([["North"],["East"],["South"],["West"]])    
g1 = enc.transform([[your_list[0][7]]]).toarray()
print(g1[0][0],g1[0][1],g1[0][2],g1[0][3])

0.0 1.0 0.0 0.0


In [79]:
enc_compass = OneHotEncoder()
enc_compass.fit([["North"],["East"],["South"],["West"]])

Y_Train = []
X_Train = []
E_Train = []
C_Train = []
for i in range(len(your_list)):
    y1 = ast.literal_eval(your_list[i][0])
    y1 = [n.strip() for n in y1]
    Y_Train.append(y1)
    
    g1 = enc_compass.transform([[your_list[i][9]]]).toarray()
    g2 = your_list[i][1]
    g3 = your_list[i][2]
    C_Train.append([g2,g3])
    x1 = [float(your_list[i][3]),float(your_list[i][4]),float(your_list[i][5]),
          float(your_list[i][6]),float(your_list[i][8]),float(your_list[i][10]),
          float(your_list[i][11]),float(your_list[i][12]),
          float(your_list[i][13]),float(your_list[i][14]),float(your_list[i][15])]
    X_Train.append(x1)
    E_Train.append(g1[0])
Y_Test = []
X_Test = []
E_Test = []
C_Test = []
for i in range(len(your_list_test)):
    y1 = ast.literal_eval(your_list_test[i][0])
    y1 = [n.strip() for n in y1]
    Y_Test.append(y1)
    
    g1 = enc_compass.transform([[your_list_test[i][9]]]).toarray()
    g2 = your_list_test[i][1]
    g3 = your_list_test[i][2]
    C_Test.append([g2,g3])
    x1 = [float(your_list_test[i][3]),float(your_list_test[i][4]),float(your_list_test[i][5]),
          float(your_list_test[i][6]),float(your_list_test[i][8]),float(your_list_test[i][10]),
          float(your_list_test[i][11]),float(your_list_test[i][12]),
          float(your_list_test[i][13]),float(your_list_test[i][14]),float(your_list_test[i][15])]
    X_Test.append(x1)
    E_Test.append(g1[0])
Y_Valid = []
X_Valid = []
E_Valid = []
C_Valid = []
for i in range(len(your_list_valid)):
    y1 = ast.literal_eval(your_list_valid[i][0])
    y1 = [n.strip() for n in y1]
    Y_Valid.append(y1)
    
    g1 = enc_compass.transform([[your_list_valid[i][9]]]).toarray()
    g2 = your_list_valid[i][1]
    g3 = your_list_valid[i][2]
    C_Valid.append([g2,g3])
    x1 = [float(your_list_valid[i][3]),float(your_list_valid[i][4]),float(your_list_valid[i][5]),
          float(your_list_valid[i][6]),float(your_list_valid[i][8]),
          float(your_list_valid[i][10]),float(your_list_valid[i][11]),float(your_list_valid[i][12]),
          float(your_list_valid[i][13]),float(your_list_valid[i][14]),float(your_list_valid[i][15])]
    X_Valid.append(x1)
    E_Valid.append(g1[0])

In [80]:
min_max_scaler = preprocessing.MinMaxScaler()
X_Train = min_max_scaler.fit_transform(X_Train)
X_Valid = min_max_scaler.fit_transform(X_Valid)
X_Test = min_max_scaler.fit_transform(X_Test)

In [81]:
enc_category = OneHotEncoder()

Out =  C_Train+C_Test+C_Valid
enc_category.fit(Out)

C_Train = enc_category.transform(C_Train).toarray()
C_Test = enc_category.transform(C_Test).toarray()
C_Valid = enc_category.transform(C_Valid).toarray()


76

In [58]:
#Out =  np.concatenate([C_Train,C_Test,C_Valid], axis=0)
Out[0]

['le_chien']

In [82]:
C_Valid.shape #(2224, 76)

(1112, 152)

In [83]:
X_Train = np.concatenate([X_Train, E_Train,C_Train], axis=1)
X_Test = np.concatenate([X_Test, E_Test,C_Test], axis=1)
X_Valid = np.concatenate([X_Valid, E_Valid,C_Valid], axis=1)


In [84]:
print(np.array(X_Train).shape,np.array(X_Test).shape, np.array(Y_Train).shape,np.array(Y_Test).shape,
      np.array(X_Valid).shape,np.array(Y_Valid).shape)

(3091, 167) (1113, 167) (3091,) (1113,) (1112, 167) (1112,)


In [4]:
def csv_generator(X_Train,Y_Train,bs,mlb,mode):
    i = 0
    while True:
        x_features = []
        y_features = []
        x = 0 
        while x < bs:
            if i >= len(X_Train):
                i = 0
                if mode == "eval":
                    break
            if i < len(X_Train):
                x_features.append(X_Train[i])
                y_features.append(Y_Train[i])
                i+=1
            else:
                i+=1
                continue
            x +=1
            
        y_features = np.array(y_features)
        y_features = mlb.transform(y_features)
        yield(np.array(x_features),y_features)

In [7]:
from keras.models import Sequential
from keras.layers import Dense
from sklearn.preprocessing import MultiLabelBinarizer
from sklearn.metrics import classification_report
from keras import backend as K


for number in range(0,10):
    train_path_geo =  "Voc2kGeometric/Training_data_"+str(number)+".csv"
    test_path_geo =  "Voc2kGeometric/Testing_data_"+str(number)+".csv"
    valid_path_geo =  "Voc2kGeometric/Validation_data_"+str(number)+".csv"

    #Read the list 
    with open(train_path_geo, 'r') as f1:
        reader = csv.reader(f1)
        your_list = list(reader)
    f1.close()

    #Read the list 
    with open(test_path_geo, 'r') as f1:
        reader = csv.reader(f1)
        your_list_test = list(reader)
    f1.close()

    #Read the list 
    with open(valid_path_geo, 'r') as f1:
        reader = csv.reader(f1)
        your_list_valid = list(reader)
    f1.close()

    enc_compass = OneHotEncoder()
    enc_compass.fit([["North"],["East"],["South"],["West"]])

    Y_Train = []
    X_Train = []
    E_Train = []
    C_Train = []
    for i in range(len(your_list)):
        y1 = ast.literal_eval(your_list[i][0])
        y1 = [n.strip() for n in y1]
        Y_Train.append(y1)

        g1 = enc_compass.transform([[your_list[i][9]]]).toarray()
        g2 = your_list[i][1]
        g3 = your_list[i][2]
        C_Train.append([g2,g3])
        x1 = [float(your_list[i][3]),float(your_list[i][4]),float(your_list[i][5]),
              float(your_list[i][6]),float(your_list[i][7]),float(your_list[i][8]),
              float(your_list[i][10]),float(your_list[i][11]),float(your_list[i][12]),
              float(your_list[i][13]),float(your_list[i][14]),float(your_list[i][15])]
        X_Train.append(x1)
        E_Train.append(g1[0])
        
    del your_list[:]
    
    Y_Test = []
    X_Test = []
    E_Test = []
    C_Test = []
    for i in range(len(your_list_test)):
        y1 = ast.literal_eval(your_list_test[i][0])
        y1 = [n.strip() for n in y1]
        Y_Test.append(y1)

        g1 = enc_compass.transform([[your_list_test[i][9]]]).toarray()
        g2 = your_list_test[i][1]
        g3 = your_list_test[i][2]
        C_Test.append([g2,g3])
        x1 = [float(your_list_test[i][3]),float(your_list_test[i][4]),float(your_list_test[i][5]),
              float(your_list_test[i][6]),float(your_list_test[i][7]),
              float(your_list_test[i][8]),float(your_list_test[i][10]),
              float(your_list_test[i][11]),float(your_list_test[i][12]),
              float(your_list_test[i][13]),float(your_list_test[i][14]),float(your_list_test[i][15])]
        X_Test.append(x1)
        E_Test.append(g1[0])
        
    del your_list_test[:]
    
    Y_Valid = []
    X_Valid = []
    E_Valid = []
    C_Valid = []
    for i in range(len(your_list_valid)):
        y1 = ast.literal_eval(your_list_valid[i][0])
        y1 = [n.strip() for n in y1]
        Y_Valid.append(y1)

        g1 = enc_compass.transform([[your_list_valid[i][9]]]).toarray()
        g2 = your_list_valid[i][1]
        g3 = your_list_valid[i][2]
        C_Valid.append([g2,g3])
        x1 = [float(your_list_valid[i][3]),float(your_list_valid[i][4]),float(your_list_valid[i][5]),
              float(your_list_valid[i][6]),float(your_list_valid[i][7]),float(your_list_valid[i][8]),
              float(your_list_valid[i][10]),float(your_list_valid[i][11]),float(your_list_valid[i][12]),
              float(your_list_valid[i][13]),float(your_list_valid[i][14]),float(your_list_valid[i][15])]
        X_Valid.append(x1)
        E_Valid.append(g1[0])
        
    del your_list_valid[:]
    
    min_max_scaler = preprocessing.MinMaxScaler()
    X_Train = min_max_scaler.fit_transform(X_Train)
    X_Valid = min_max_scaler.fit_transform(X_Valid)
    X_Test = min_max_scaler.fit_transform(X_Test)
    
    enc_category = OneHotEncoder()
    Out =  C_Train+C_Test+C_Valid
    enc_category.fit(Out)
    
    del Out[:]
    
    C_Train = enc_category.transform(C_Train).toarray()
    C_Test = enc_category.transform(C_Test).toarray()
    C_Valid = enc_category.transform(C_Valid).toarray()


    X_Train = np.concatenate([X_Train, E_Train,C_Train], axis=1)
    E_Train = []
    C_Train = []
    X_Test = np.concatenate([X_Test, E_Test,C_Test], axis=1)
    E_Test = []
    C_Test = []
    X_Valid = np.concatenate([X_Valid, E_Valid,C_Valid], axis=1)
    E_Valid = []
    C_Valid = []
    
    mlb = MultiLabelBinarizer()
    mlb.fit_transform(Y_Train)
    mlb.fit_transform(Y_Valid)
    mlb.fit_transform(Y_Test)
    
    model = Sequential([
        Dense(2048, init="uniform",activation='relu', input_shape=(168,)),
        Dense(1024, kernel_initializer="uniform" ,activation='relu'),
        Dense(len(mlb.classes_), activation='sigmoid'),
    ])

    trainGen = csv_generator(X_Train,Y_Train,32,mlb,"train")
    valGen = csv_generator(X_Valid,Y_Valid,32,mlb,"train")

    model.compile(optimizer='adam',
                  loss='binary_crossentropy',
                  metrics=['categorical_accuracy'])

    hist = model.fit_generator(trainGen, steps_per_epoch = (len(X_Train)/32)-1,validation_data=valGen, validation_steps=(len(X_Valid)/32)-1,epochs=30)
    model_name = "GeometricModels/SV"+str(number)+"_.h5"
    model.save(model_name)
    Y_Test = mlb.transform(Y_Test)
    predictions = model.predict(X_Test)
    write_Accuracy(predictions,Y_Test,number,mlb)

    K.clear_session()

/home/nuclearvodka/.local/lib/python3.6/site-packages/ipykernel_launcher.py:133: UserWarning: Update your `Dense` call to the Keras 2 API: `Dense(2048, activation="relu", input_shape=(168,), kernel_initializer="uniform")`


Epoch 1/30
97/96 [==============================] - 1s 13ms/step - loss: 0.2764 - categorical_accuracy: 0.1537 - val_loss: 0.2370 - val_categorical_accuracy: 0.3107
Epoch 2/30
97/96 [==============================] - 0s 5ms/step - loss: 0.2155 - categorical_accuracy: 0.2677 - val_loss: 0.2267 - val_categorical_accuracy: 0.3493
Epoch 3/30
97/96 [==============================] - 0s 5ms/step - loss: 0.1960 - categorical_accuracy: 0.2974 - val_loss: 0.2213 - val_categorical_accuracy: 0.3502
Epoch 4/30
97/96 [==============================] - 0s 5ms/step - loss: 0.1818 - categorical_accuracy: 0.3260 - val_loss: 0.2211 - val_categorical_accuracy: 0.3428
Epoch 5/30
97/96 [==============================] - 0s 5ms/step - loss: 0.1708 - categorical_accuracy: 0.3428 - val_loss: 0.2220 - val_categorical_accuracy: 0.3290
Epoch 6/30
97/96 [==============================] - 0s 5ms/step - loss: 0.1613 - categorical_accuracy: 0.3586 - val_loss: 0.2257 - val_categorical_accuracy: 0.3244
Epoch 7/30
97/9

/home/nuclearvodka/.local/lib/python3.6/site-packages/ipykernel_launcher.py:133: UserWarning: Update your `Dense` call to the Keras 2 API: `Dense(2048, activation="relu", input_shape=(168,), kernel_initializer="uniform")`


Epoch 1/30
97/96 [==============================] - 1s 8ms/step - loss: 0.2745 - categorical_accuracy: 0.1498 - val_loss: 0.2368 - val_categorical_accuracy: 0.2013
Epoch 2/30
97/96 [==============================] - 0s 5ms/step - loss: 0.2152 - categorical_accuracy: 0.2600 - val_loss: 0.2226 - val_categorical_accuracy: 0.2059
Epoch 3/30
97/96 [==============================] - 0s 4ms/step - loss: 0.1954 - categorical_accuracy: 0.2909 - val_loss: 0.2189 - val_categorical_accuracy: 0.2040
Epoch 4/30
97/96 [==============================] - 0s 5ms/step - loss: 0.1823 - categorical_accuracy: 0.3061 - val_loss: 0.2183 - val_categorical_accuracy: 0.2068
Epoch 5/30
97/96 [==============================] - 0s 5ms/step - loss: 0.1719 - categorical_accuracy: 0.3351 - val_loss: 0.2180 - val_categorical_accuracy: 0.2151
Epoch 6/30
97/96 [==============================] - 0s 5ms/step - loss: 0.1635 - categorical_accuracy: 0.3512 - val_loss: 0.2215 - val_categorical_accuracy: 0.2233
Epoch 7/30
97/96

/home/nuclearvodka/.local/lib/python3.6/site-packages/ipykernel_launcher.py:133: UserWarning: Update your `Dense` call to the Keras 2 API: `Dense(2048, activation="relu", input_shape=(168,), kernel_initializer="uniform")`


...

Epoch 1/30
96/95 [==============================] - 1s 7ms/step - loss: 0.2741 - categorical_accuracy: 0.1527 - val_loss: 0.2299 - val_categorical_accuracy: 0.2634
Epoch 2/30
96/95 [==============================] - 0s 4ms/step - loss: 0.2158 - categorical_accuracy: 0.2614 - val_loss: 0.2191 - val_categorical_accuracy: 0.2946
Epoch 3/30
96/95 [==============================] - 0s 4ms/step - loss: 0.1968 - categorical_accuracy: 0.3047 - val_loss: 0.2143 - val_categorical_accuracy: 0.3098
Epoch 4/30
96/95 [==============================] - 0s 4ms/step - loss: 0.1834 - categorical_accuracy: 0.3229 - val_loss: 0.2154 - val_categorical_accuracy: 0.3214
Epoch 5/30
96/95 [==============================] - 0s 4ms/step - loss: 0.1726 - categorical_accuracy: 0.3451 - val_loss: 0.2185 - val_categorical_accuracy: 0.3098
Epoch 6/30
96/95 [==============================] - 0s 5ms/step - loss: 0.1641 - categorical_accuracy: 0.3617 - val_loss: 0.2240 - val_categorical_accuracy: 0.3214
Epoch 7/30
96/95

/home/nuclearvodka/.local/lib/python3.6/site-packages/ipykernel_launcher.py:133: UserWarning: Update your `Dense` call to the Keras 2 API: `Dense(2048, activation="relu", input_shape=(168,), kernel_initializer="uniform")`


Epoch 1/30
98/97 [==============================] - 1s 7ms/step - loss: 0.2719 - categorical_accuracy: 0.1244 - val_loss: 0.2376 - val_categorical_accuracy: 0.2254
Epoch 2/30
98/97 [==============================] - 0s 4ms/step - loss: 0.2157 - categorical_accuracy: 0.2545 - val_loss: 0.2295 - val_categorical_accuracy: 0.2235
Epoch 3/30
98/97 [==============================] - 0s 4ms/step - loss: 0.1964 - categorical_accuracy: 0.2883 - val_loss: 0.2274 - val_categorical_accuracy: 0.2178
Epoch 4/30
98/97 [==============================] - 0s 4ms/step - loss: 0.1828 - categorical_accuracy: 0.3166 - val_loss: 0.2282 - val_categorical_accuracy: 0.2320
Epoch 5/30
98/97 [==============================] - 0s 4ms/step - loss: 0.1706 - categorical_accuracy: 0.3409 - val_loss: 0.2299 - val_categorical_accuracy: 0.2443
Epoch 6/30
98/97 [==============================] - 0s 4ms/step - loss: 0.1612 - categorical_accuracy: 0.3670 - val_loss: 0.2349 - val_categorical_accuracy: 0.2453
Epoch 7/30
98/97

/home/nuclearvodka/.local/lib/python3.6/site-packages/ipykernel_launcher.py:133: UserWarning: Update your `Dense` call to the Keras 2 API: `Dense(2048, activation="relu", input_shape=(168,), kernel_initializer="uniform")`


Epoch 1/30
96/95 [==============================] - 1s 7ms/step - loss: 0.2764 - categorical_accuracy: 0.1553 - val_loss: 0.2305 - val_categorical_accuracy: 0.1691
Epoch 2/30
96/95 [==============================] - 0s 4ms/step - loss: 0.2173 - categorical_accuracy: 0.2744 - val_loss: 0.2193 - val_categorical_accuracy: 0.1811
Epoch 3/30
96/95 [==============================] - 0s 4ms/step - loss: 0.1972 - categorical_accuracy: 0.3138 - val_loss: 0.2172 - val_categorical_accuracy: 0.1949
Epoch 4/30
96/95 [==============================] - 0s 4ms/step - loss: 0.1827 - categorical_accuracy: 0.3389 - val_loss: 0.2186 - val_categorical_accuracy: 0.2050
Epoch 5/30
96/95 [==============================] - 0s 5ms/step - loss: 0.1716 - categorical_accuracy: 0.3604 - val_loss: 0.2230 - val_categorical_accuracy: 0.2160
Epoch 6/30
96/95 [==============================] - 0s 5ms/step - loss: 0.1631 - categorical_accuracy: 0.3753 - val_loss: 0.2285 - val_categorical_accuracy: 0.2178
Epoch 7/30
96/95

In [1]:
def y_count_1(a):
    m = len(a)
    y_count = []
    for i in range(m):
        #print(a[i])
        if a[i] == 1:
            y_count.append(i)
    return y_count

def get_pred_rank1(a):
    N=len(a)
    b=np.zeros([N],dtype=[('prob',float),('idx',int)])
    #for i in range(N)
    b['idx']=np.arange(N)
    b['prob']=np.copy(a)
    b.sort(order='prob')
    return b

def TP_FN(y_pred,y_test,k):
    TP = []
    FN = []
    for j in range(len(y_pred)):
        Y = y_test[j]
        H = get_pred_rank1(y_pred[j])        
        y_count = y_count_1(y_test[j])
        y_rank = np.flip(np.asarray(H['idx'],dtype=int),0) 
        #print(y_rank)
        for p in y_count:
            if p in y_rank[:k]:
                TP.append(p)
            else:
                #print(p)
                FN.append(p)
            
    return TP,FN 

def get_per_label(TP,FN,n):
    Recalls = []
    for i in range(0,n):
        TP1 = TP.get(i)
        FN1 = FN.get(i)
        if TP1 == None:
            TP1 = 0
        if FN1 == None:
            FN1 = 0
                
        if (TP1+FN1) == 0:
            Recall = 0
        else:
            Recall  = (TP1)/(TP1+FN1)
            
        Recalls.append(Recall)
        
    return Recalls

#Recall => TP/TP+FN
def get_recalls_for_k(predictions,Y_Test,k,n):
    ALL_Recalls = []
    for i in range(k): 
        TP,FN  = TP_FN(predictions,Y_Test,i+1)
        TP_Counter = Counter(TP)
        FN_Counter = Counter(FN)
        Recall = get_per_label(TP_Counter,FN_Counter,n)
        ALL_Recalls.append(Recall)
    return ALL_Recalls

In [5]:
def write_Accuracy(predictions,Y_Test,number,mlb):
    recalls = get_recalls_for_k(predictions,Y_Test,4,len(mlb.classes_))
    
    row = ["Classes","k = 1 ","k = 2","k = 3","k = 4"]
    rows = []
    rows.append(row)
    for i in range(len(mlb.classes_)):
        row  = [mlb.classes_[i],recalls[0][i],recalls[1][i],recalls[2][i],recalls[3][i]]
        rows.append(row)    
    
    rows.append(["Micro AVG",sum(recalls[0])/len(recalls[0]),sum(recalls[1])/len(recalls[1]),sum(recalls[2])/len(recalls[2]),sum(recalls[3])/len(recalls[3])])
    location = "GeometricReports/SV"+str(number)+".csv"
    with open(location,"a",newline="") as f1: 
        cw = csv.writer(f1)
        cw.writerows(rows)
    f1.close()
    recalls = []
    rows = []
    predictions = []
    Y_Test = []